### notes
1. нужно ли хранить в выходном массиве непосредственно сами фичи? или отрисовывать боксы на изображениях сразу, не выходя из цикла? 
2. что делать с объектами, которые пропадают из поля зрения?
3. **что делать с объектами, для которых нашлась всего одна фича? просто удалять?** (есть прикольная мысль о том, как связан объем изображения с количеством найденных фич - мб есть смысл отбрасовать объекты еще на этапе выделения фрагмента изображения)
4. все-таки вопрос об идеальной структуре данных остается неотвеченным (хотя **надо просто попробовать разные**)

In [1]:
import os
from cv2 import cv2
import yaml
import re
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

In [2]:
# параметра отображения картинок внутри блокнота
plt.rcParams['figure.figsize'] = [13, 7]

### Вспомогательные функции

In [3]:
def readYAMLFile(file):
    ret = {}
    skip_lines = 1    # Skip the first line which says "%YAML:1.0". Or replace it with "%YAML 1.0"
    with open(file) as fin:
        for i in range(skip_lines):
            fin.readline()
        yamlFileOut = fin.read()
        myRe = re.compile(r":([^ ])")   # Add space after ":", if it doesn't exist. Python yaml requirement
        yamlFileOut = myRe.sub(r': \1', yamlFileOut)
        ret = yaml.safe_load(yamlFileOut)
    return ret

In [4]:
# расстояние между двумя точками
def dist(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

In [292]:
# оставляем только нужные ключи
def convert_object(image, obj):
    # converted = {key: obj[key] for key in ['x_min', 'y_min', 'x_max', 'y_max']}
    # converted['flow'] = []
    
    converted = {
        'id': obj['id'],
        'flow': [
            {key: obj[key] for key in ['x_min', 'y_min', 'x_max', 'y_max']}
        ]
        
    }
    
    converted['flow'][-1]['frame_id'] = image[18:-4]
    
    return converted

In [114]:
def get_frame(object_, p0, p1):
    
    # центр рамки объекта на первом кадре
    center_1 = np.array([(object_['flow'][-1]['x_min'] + object_['flow'][-1]['x_max']) / 2, 
                         (object_['flow'][-1]['y_min'] + object_['flow'][-1]['y_max']) / 2], 
                                                                                    dtype='float32')
    
    
    centroid_1, centroid_2 = [x.mean(axis=-2) for x in [p0, p1]]
    
    
    dist_between_points_1 = np.array([dist(p0[i], p0[j]) for i in range(len(p0)) 
                                                for j in range(i + 1, len(p0))]).mean()
    dist_between_points_2 = np.array([dist(p1[i], p1[j]) for i in range(len(p1)) 
                                                for j in range(i + 1, len(p1))]).mean()
    
    
    scale = dist_between_points_1 / dist_between_points_2
    difference = (center_1 - centroid_1) / scale
        
    # параметры новой рамки
    center_2 = centroid_2 + difference
    width_2, height_2 = (object_['flow'][-1]['x_max'] - object_['flow'][-1]['x_min']) / scale, \
                        (object_['flow'][-1]['y_max'] - object_['flow'][-1]['y_min']) / scale
            
        
    # итог: координаты предсказанной рамки    
    x_min = int(round(center_2[0] - width_2 / 2))
    y_min = int(round(center_2[1] - height_2 / 2))
    x_max = int(round(center_2[0] + width_2 / 2))
    y_max = int(round(center_2[1] + height_2 / 2))
        
        
    return {'x_min': x_min,'y_min': y_min,'x_max': x_max,'y_max': y_max}
    

In [131]:
# intersection over union
def get_intersection(box_1, box_2):
    
    # координаты прямоугольника - пересечения
    x_min = max(box_1['x_min'], box_2['x_min'])
    y_min = max(box_1['y_min'], box_2['y_min'])
    x_max = min(box_1['x_max'], box_2['x_max'])
    y_max = min(box_1['y_max'], box_2['y_max'])
    
    
    if x_max < x_min or y_max < y_min:
        return -1
    
    
    intersection_area = (x_max - x_min) * (y_max - y_min)
    box_1_area = (box_1['x_max'] - box_1['x_min']) * (box_1['y_max'] - box_1['y_min'])
    box_2_area = (box_2['x_max'] - box_2['x_min']) * (box_2['y_max'] - box_2['y_min'])
    
    return intersection_area / (box_1_area + box_2_area - intersection_area)

### Данные

In [396]:
imgs_path = 'images/cam0/'
frms_path = 'frames/cam0/'

images = sorted([imgs_path + x for x in os.listdir(imgs_path) if x.endswith('.png')])
frames = sorted([frms_path + x for x in os.listdir(frms_path) if x.endswith('.yml')])

### Оптический поток
+ в данном случае используем SIFT (вообще говоря, не единственный возможный вариант)

In [8]:
# lucas-kanade parameters
lk_params = dict(winSize  = (15, 15),
                 maxLevel = 2,
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

sift = cv2.SIFT_create()

In [294]:
images = images[:6]
frames = frames[:6]

In [397]:
# объекты, которые находятся на рассматриваемом кадре
objects = []

# объекты, которые ушли с кадра
gone = []

# пары объектов "бокс с оптического потока" -> "бокс с нейронки", у которых iou > 0.5
merged = []


while len(images) > 1:
    
    # 1 и 2 кадр
    old_image, image = [cv2.imread(images[i], cv2.IMREAD_COLOR) for i in range(2)]
    old_gray, gray = [cv2.cvtColor(x, cv2.COLOR_BGR2GRAY) for x in [old_image, image]]
    
    # условие для 0 шага
    if not objects:
        # за исходный набор объектов берем предсказания нейросети для 1 кадра
        objects = [convert_object(images[0], x) for x in readYAMLFile(frames[0])['boxes']]
    
    # набор боксов, предсказанный нейронкой для следующего кадра
    new_neuro_objects = [x for x in readYAMLFile(frames[1])['boxes']]
    
    
    for obj in objects:
        # вырезанный кусок изображения, содержащий объект obj
        cut = old_gray[obj['flow'][-1]['y_min']:obj['flow'][-1]['y_max'], \
                           obj['flow'][-1]['x_min']:obj['flow'][-1]['x_max']]
           
        
        # набор сифтовых фичей для объекта obj
        p0 = cv2.KeyPoint_convert(sift.detect(cut, None))
        
        
        # отлов случая с всего одной найденной фичой
        # далее этот объект не рассматривается
        if len(p0) <= 2:
            gone.append(obj)
            objects.remove(obj)
            break
        
        p0 = np.array(p0 + (obj['flow'][-1]['x_min'], obj['flow'][-1]['y_min']), dtype='float32')
        
        
        # набор фичей для следующего кадра, посчитанные оптическим потоком
        # (вылетает Assertion error, когда в p0 лежит всего одна фича)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)
        p1 = p1[np.all(st==1, axis=-1)]
        
        
        # получение набора координат для нового бокса
        obj['flow'].append(get_frame(obj, p0, p1))
        obj['flow'][-1]['frame_id'] = images[1][18:-4]
       
        
        # блок сравнения списка объектов с оптического потока и с нейронки
        for new in new_neuro_objects:
            k = get_intersection(new, obj['flow'][-1])
            if k > 0.7:
                merged.append((obj['id'], new['id'], images[1][18:-4]))
                new_neuro_objects.remove(new)
    
    
    # блок перехода на следующий шаг
    
    objects += [convert_object(images[1], x) for x in new_neuro_objects]
    images = images[1:]
    frames = frames[1:]

### проблемы, которые я вижу: 
1. почему отработавшие объекты (длины = 1) не удаляются из списка objects
2. что творится в блоке сравнения потока и нейронки???
3. почему поток работает так плохо? (всего 10 объектов из 303 с длиной потока > 3)

In [403]:
len(objects) + len(gone)

303

In [401]:
good_to_draw = [x for x in gone if len(x['flow']) > 3]

In [402]:
len(good_to_draw)

10

In [374]:
def draw_visualisation(obj):
    for x in obj['flow']:
        id_ = x['frame_id']
        img = cv2.imread(f'images/cam0/frame-{id_}.png', cv2.IMREAD_COLOR)
        cv2.rectangle(img, (x['x_min'], x['y_min']), (x['x_max'], x['y_max']), (0, 255, 0), 3)
        cv2.imwrite(f'visualisation/frame-{id_}.png', img)

In [387]:
draw_visualisation(good_to_draw[3])